# Pytorch tutorial 3

## Softmax classifier
One Hot Encoding....

In [1]:
import torch
from torch.autograd import Variable
torch.manual_seed(777)

In [2]:
x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5],
          [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0],
          [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]  # 0번 index : 1, 1번 index : 1, 2번 index : 1 (3가지 해답만 존재)

    텐서플로의 경우
    X = tf.placeholder("float", [None, 4])
    Y = tf.placeholder("float", [None, 3])  # One Hot Encoding
    nb_classes = 3

In [3]:
X = Variable(torch.Tensor(x_data))
Y = Variable(torch.Tensor(y_data))  # One Hot Encoding
nb_classes = 3

    [텐서플로] : 함수를 정의하면, 아래서 받아서 연관함수를 정의하고, 또 받아서 연관함수를.... (bottom up 방식)

    hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [4]:
# 개별 함수를 개별 정의 후, model에서 연결이 용이 (top down 방식)
softmax = torch.nn.Softmax()
linear = torch.nn.Linear(4, nb_classes, bias=True)
model = torch.nn.Sequential(linear, softmax)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

    # Cross entropy cost/loss
    # [tensorflow] tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
    # [pytorch]    torch.sum(-Y * torch.log(hypothesis) , 1).mean()

In [5]:
for step in range(2001):
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = -Y * torch.log(hypothesis)
    cost = torch.sum(cost, 1).mean()
    cost.backward()
    optimizer.step()
    if step % 400 == 0:
        print('step : ', step, 'cost :', cost.data.numpy())

step :  0 cost : [ 3.88193607]
step :  400 cost : [ 0.5211699]
step :  800 cost : [ 0.33575076]
step :  1200 cost : [ 0.22303778]
step :  1600 cost : [ 0.18638299]
step :  2000 cost : [ 0.15975267]


#### Testing & One-hot encoding 1
[[1, 11, 7, 9]] tensor 데이터로 model 을 Test ---> Y^ = 1 로 예측

In [6]:
a = model(Variable(torch.Tensor([[1, 11, 7, 9]])))
torch.max(a, 1)#[0].data.numpy()

(Variable containing:
  0.9917
 [torch.FloatTensor of size 1x1], Variable containing:
  1
 [torch.LongTensor of size 1x1])

In [7]:
torch.max(a, 1)[0]#.data.numpy()

Variable containing:
 0.9917
[torch.FloatTensor of size 1x1]

In [8]:
torch.max(a, 1)[1]#.data.numpy()

Variable containing:
 1
[torch.LongTensor of size 1x1]

In [9]:
torch.max(a, 1)[0].data.numpy()

array([[ 0.991687]], dtype=float32)

In [10]:
torch.max(a, 1)[1].data.numpy()

array([[1]])

In [11]:
a.data.numpy(), torch.max(a, 1)[1].data.numpy()

(array([[  8.30345508e-03,   9.91687000e-01,   9.57232714e-06]], dtype=float32),
 array([[1]]))

#### Testing & One-hot encoding 2
[[1, 3, 4, 3]] tensor 데이터로 model 을 Test  ---> Y^ = 0 로 예측

In [12]:
b = model(Variable(torch.Tensor([[1, 3, 4, 3]])))
b.data.numpy(), torch.max(b, 1)[1].data.numpy()

(array([[ 0.82575417,  0.15875565,  0.01549017]], dtype=float32), array([[0]]))

#### Testing & One-hot encoding 3
[[1, 1, 0, 1]] tensor 데이터로 model 을 Test  ---> Y^ = 2 로 예측

In [13]:
c = model(Variable(torch.Tensor([[1, 1, 0, 1]])))
c.data.numpy(), torch.max(c, 1)[1].data.numpy()

(array([[  1.63065614e-08,   3.70370341e-04,   9.99629617e-01]], dtype=float32),
 array([[2]]))

#### Testing & One-hot encoding 1
test1, test2, test3 tensor 데이터를 model 에 함께 Test

In [14]:
all = model(Variable(torch.Tensor([[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]])))
all.data.numpy(), torch.max(all, 1)[1].data.numpy()

(array([[  8.30345508e-03,   9.91687000e-01,   9.57232714e-06],
        [  8.25754166e-01,   1.58755645e-01,   1.54901687e-02],
        [  1.63065614e-08,   3.70370341e-04,   9.99629617e-01]], dtype=float32),
 array([[1],
        [0],
        [2]]))

## Softmax classifier
데이터로 동물 식별하기....

In [15]:
import torch
from torch.autograd import Variable
import numpy as np
torch.manual_seed(777)  # for reproducibility

In [16]:
# Predicting animal type based on various features
xy = np.loadtxt('./Lecture/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
print(x_data[::30])
print('x_data :', x_data.shape,'\n\n',
      'y_data set :',set([i[0] for i in y_data]), '\ny_data :' , y_data.shape, )

nb_classes = 7  # 0 ~ 6  ( 총 7개 동물로 분류된 데이터)

[[ 1.  0.  0.  1.  0.  0.  1.  1.  1.  1.  0.  0.  4.  0.  0.  1.]
 [ 0.  0.  1.  0.  1.  0.  0.  0.  0.  1.  0.  0.  6.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  1.  1.  1.  1.  0.  0.  1.  0.  1.  0.  1.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  1.  1.  0.  0.  4.  1.  0.  1.]]
x_data : (101, 16) 

 y_data set : {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0} 
y_data : (101, 1)


In [17]:
# one hot encoding (0~6 의 숫자로 되어있는 'y_data'를   'One Hot Encoding 포맷'으로 변환)
X = Variable(torch.from_numpy(x_data))
Y = Variable(torch.from_numpy(y_data))

Y_one_hot = torch.zeros(Y.size()[0], nb_classes)  # [0,0,0,0,0,0] 배경 데이터 만들기
Y_one_hot.scatter_(1, Y.long().data, 1)           # 1 ~ 6 데이터에 해당 위치에 1을 찍는다
Y_one_hot = Variable(Y_one_hot)
print("one_hot", Y_one_hot.data[::20])

one_hot 
    1     0     0     0     0     0     0
    0     1     0     0     0     0     0
    0     0     0     0     0     1     0
    0     0     0     1     0     0     0
    0     0     1     0     0     0     0
    0     1     0     0     0     0     0
[torch.FloatTensor of size 6x7]



In [18]:
softmax = torch.nn.Softmax()
model = torch.nn.Linear(16, nb_classes, bias=True)  # 독립변수 데이터 : 16 컬럼, 출력변수 : nb_classes(one-hot encoding) 

# Cross entropy cost/loss
criterion = torch.nn.CrossEntropyLoss()    # Softmax is internally computed.
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [19]:
# 학습모델 만들기
for step in range(2001):
    optimizer.zero_grad()
    hypothesis = model(X)
    # Label has to be 1D LongTensor
    cost = criterion(hypothesis, Y.long().view(-1))
    cost.backward()
    optimizer.step()
    prediction = torch.max(softmax(hypothesis), 1)[1].float()
    correct_prediction = (prediction.data == Y.data)
    accuracy = correct_prediction.float().mean()

    if step % 400 == 0:
        print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, cost.data[0], accuracy))

Step:     0	Loss: 1.971	Acc: 5.94%
Step:   400	Loss: 0.211	Acc: 95.05%
Step:   800	Loss: 0.125	Acc: 99.01%
Step:  1200	Loss: 0.089	Acc: 100.00%
Step:  1600	Loss: 0.069	Acc: 100.00%
Step:  2000	Loss: 0.056	Acc: 100.00%


In [20]:
# 학습한 모델을 테스트 해보자 (거의 대부분 일치한 결과가 도출)
pred = torch.max(softmax(hypothesis), 1)[1].float(); i = 0
for p, y in zip(pred, Y): 
    i += 1
    if i % 7 == 0:
        print("[{}] Prediction: {} True Y: {}".format(bool(p.data[0] == y.data[0]), p.data.int()[0], y.data.int()[0]))

[True] Prediction: 0 True Y: 0
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 2 True Y: 2
[True] Prediction: 0 True Y: 0
[True] Prediction: 2 True Y: 2
[True] Prediction: 1 True Y: 1
[True] Prediction: 2 True Y: 2
[True] Prediction: 5 True Y: 5


In [22]:
pred = torch.max(softmax(hypothesis), 1)[1].float()
for i in range(30002):
    if i % 5000 == 0:
        print(i)
    for p, y in zip(pred, Y): 
        if p.data[0] != y.data[0]:
            print("[{}] Prediction: {} True Y: {}".format(bool(p.data[0] == y.data[0]), p.data.int()[0], y.data.int()[0]))

0
5000
10000
15000
20000
25000
30000
